![title](../data/imgs/boosting13.png)

Como vimos, Gradient Boosted Decision Trees é uma tecnica de Boosting que utiliza Stumped Decision Trees para a melhora gradual do modelo.

Mas como que isso é feito?

![title](../data/imgs/boosting12.png)

Esse ciclo é repetido de formas sucessivas enquanto construímos novos modelos e fazemos a devida combinação com nosso modelo de `Ensemble`. Nós iniciamos o ciclo ao calculuar os erros para cada valor do nosso dataset de treino. Então, nós construímos um novo modelo para fazer a previsão em cima desses erros e adicionamos essa previsão-de-erro ao nosso "ensemble de modelo".

Para fazer a previsão, nós adicionamos as predições de todos os modelos anteriores. Essas previsões são usadas para calcular novos modelos e então, as acoplamos no nosso ensemble.

Confuso? Para entender um pouco melhor a ideia, vamos relembrar a ideia de Regressão Linear

## Relembrando Regressão Linear

Na aula de Regressão Linear, vocês viram um método analítico (dos mínimos quadrados). Ele funciona bem com datasets pequeno, mas temos alguns problemas como datasets maiores. 

Na regressão, quando queremos definir a curva que define nosso modelo, nós podemos fazê-lo por meio do seguinte algoritmo:

- Ter um modelo $F_1(x)$ que faça a predição dos nossos dados
- Medir os nossos resíduos (erros): $h(x) = y - F(x)$
- Criar um novo modelo: $F_2(x) = F_1(x) + h_1(x)$

Podemos repetir isso $M$ vezes de forma a irmos combinando cada vez mais modelos:
![title](../data/imgs/boosting14.png)

Só que nós queremos minimizar os erros. Podemos fazer isso definindo uma função de custo que define o nosso erro, por exemplo: 

$f(x) = \sum_{i=1}^N{L(y_i,h(x_i))}$, ou seja, todos os nossos dados de treinamento. Aqui, então, $L$ pode ser o erro quadrático $L = (y_i - h(x_i))$

E para minimizar nós introduzimos a ideia de gradiente:

Para um dado ponto x do nosso dataset, queremos reduzí-lo ao mínimo local:

![title](../data/imgs/boosting15.png)


Ou seja, queremos fazer o nossa previsão ter seu erro cada vez mais reduzido, de forma que ele seja **muito próximo** de 0.

Matemátiquês a parte, o que está acontecendo é basicamente isso:

$x_i = x_{i-1} - \eta\frac{d_f(x_{i-1})}{dx}$, em que $\eta$ representa a _intensidade_ que queremos fazer esse ajuste em função do erro

![](../data/imgs/gradient_descent_example.gif)

A diferença do GD para Gradient Boost é que no GD vamos atualizando ponto a ponto, por interação. No caso do Gradient Boosting, nós não temos uma forma clara da forma da nossa função: ela é apenas um amontoado de árvore, em que cada árvore é atualizada pr interação. No caso do Gradient Descent, após a iteração _i_, você têm a atualização do ponto _i_ analizado. Para árvores, após a _i_ ésima interação, teremos _i_ árvores.

## O Dataset

Vamos dar uma explorada no nosso dataset

O que ele quer dizer para a gente?


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

In [9]:
data = pd.read_csv('../data/housing/inputs/train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [10]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [11]:
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [ ]:
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

In [ ]:
my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [5]:
import xgboost as xgb

In [17]:
my_model = xgb.XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [18]:
# make predictions
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 15966.5242295


In [19]:
my_model.score(test_X, test_y)

0.88673547217612436

## Seleção de Hiperparâmetros

O XGBoost têm alguns hiperparâmetros que podem afetar muito a acurácia do seu modelo e velocidade de treinamento.

Os dois primeiros são:
 - n_estimators 
 - early_stopping_rounds

#### Número de estimadores

Número de estimadores podem resultar em `underfitting` enquanto números elevados podem levar a `overfitting`. Novamente, caímos aqui no dilema entre complexidade e generalização de treinamento. Você deve experimentar com o dataset para achar os valores ideiais. Um número mágico (para ajudar) variam de 100 a 1000 mas depende muito do __learning rate__, que já vamos falar.

#### Early stopping rounds

Esse argumento, por sua vez, fornece uma forma de encontrar o valor ideal automaticamente. Ele faz com que o modelo pare de iteragir quando o score de validação seja concluido, mesmo que o número de estimadores não tenha atingido o número ideal. Aqui, é ideal usar um valor alto para o número de estimadores e, então, usar o early_stopping_rounds para encontrar um tempo ótimo para parar de iterar. Como por aleatoriedade, o val score pode não melhoreas, queremos um número de rounds seguidos em que não há melhora para considerarmos que nosso modelo deva parar.
Um valor bom aqui é val_score = 5


In [22]:
#Use 5 early stop roungs, n estimator as 1000
# early stop belongs to fit and n estimator belongs to 1000
my_model = xgb.XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

#### learning_rate

Aqui é exatamente o **$\eta$** que vimos lá em cima quando falamos do Gradient Descent. E, como ele determina a intensidade, ele é uma das peças chaves para atingir os melhores modelos do XGBoost

Na prática, ele serve para controlar o _overfitting_ do nosso modelo.

Assim, não precisamos nos preocupar muito com um número máximo de árvores. Se usarmos early_stopping, o número apropriado de árvores vai ser encontrado de forma automática, evitando o overfit.

Sendo assim, é melhor um learning rate alto ou baixo?

In [23]:
#based on this, complete, including learning_rate
my_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

#### n_jobs

Em datasets grandes, em que o tempo de execução é levado em consideração, você pode fazer uso de paralelism para que seus modelos sejam executados mais rapidamente. É comum setar o número de jobs (n_jobs) exatamente igual ao número de cores na sua máquina. Em datasets pequenos, contudo, isso não faz muita diferença

O XGBoost tem vários outros parâmetros que valem a pena serem discutidos, mas para um primeiro momento, esses são os ideais.

#### objective_functions

Objective Functions (ou Loss Functions) são as funções que vimos nos exemplos. Elas quantificam o quão longe nossa predição está do resultado real

Em outras palavras, queremos, minimizar essa função

No xgboost elas recebem os seguintes nomes:
 - reg:linear para problemas de regressão
 - reg:logistic usados em problemas de classificação quando você só quer a classe, não a chance de ser ela
 - reg: binary usados em problemas de classificacao quando você quer a chance de pertencer a cada classe
 